<a href="https://colab.research.google.com/github/parkerburchett/EconCapstone/blob/main/Notebook_to_Gather_Ethermine_Transaction_data_and_do_Cleaning_and_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is the first draft of the notebook I am writing for my Economics Capstone to get a published paper. \

https://www.etherchain.org/charts Look at "mining revenue" gives USD value per day of a GH/s. Use this instead of building it yourself.
### Outline: What will this notebook do?

1. Use the Etherscan.io charts to convert into production 
* Block Number to Date.(Year, Month)
* (Year, Month) to Monthly average Value of 1 day of mining at 1 GH/s (Etherscan.io chart). Just average by month. 

* (Year, Month) to Average ETH Price. 


* You will need a dictionary that looks like 
(Year, Month): Average Gh/s needed to mine make 1 ETH 
### Called production dictionary. 

2. You will need to write custom Estimate_hashrate 

*Pseudocode:
  Estimate_HashRate(Year, Month, ETH earned this month):

  estimated_hashrate_in_ghS = Production_Dictionary[(Year, Month) * ETH earned  this month)


3. Use the Etherscan.io API to get all of the transactions from 5 large pools, and store that in pyspark on Google Colab. You will need to save that to someplace in the cloud so that you can get it later and put the public link. I think drop box is the right place for it.

 -> What I want the data to look like.
 This information is factual raw data. 
(Block Number of Transaction, Pool Address, Miner Address, Amount of ETH)


Every Single Transaction will be mapped into this form. 

(Block Number, Day, Year, Month, Pool Address, Miner Address, Amount of ETH, Estimated_hashrate)

This data will then be grouped by Miner_address and Month.

Intermediate(after group by) result:

Month, Year, Pool Address, Miner Address, ETH Earned this month, Estimated Hashrate, (ETH earned this month * Average Monthly Price of ETH) as monthly USD Revenue.

Frame work for Miner Features
This is how I am choosing to categorize Miners. 

## Miner Features
Start month ( cast this as an int with January 2015 as 1). # month of first  income).
End Month (cast this as an int with March 2021 as the 80 or whatever the number happens to be). Last month with income. or + infinity if that was march 2021. Age as number of full months,
Median Hashrate, as size. 



Dummy Varibles for Pool address, 
Dummy Variables for if they went to a different




Framework for OLS regression:

##Independent variable: 
* Estimated Monthly Hashrate.

###Dependent Variables
Miner Features. 
Year
Month,
Average ETH price,
average Gh/s value



### Setup Pyspark and Connect it to your google drive

In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Open Connection to Google drive

from google.colab import drive
drive.mount('/content/drive')
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
print("\n\n DRIVE SETUP")

     |████████████████████████████████| 212.3MB 61kB/s 
     |████████████████████████████████| 204kB 45.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=da43bf282e6ba0cba66d82a169f295f6b97ccd0894d98b53fbf72ed729a6713c
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 30 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

## Install Libraries 

In [78]:
import pandas as pd
import numpy as np
import json
import requests
import math
import datetime
import matplotlib.pyplot as plt
import time
%matplotlib inline

import pyspark
from functools import reduce
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import DataType
from pyspark import SparkContext, SparkConf


### Create Spark Session

In [4]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050") # copy and pasted this, dont' know what it means
# create the context
# try catch block lets you start again from the top without causing problems. 
try:
  sc.stop()
  sc = pyspark.SparkContext(conf=conf)
except:
  sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

print('Successfully Created Spark Session')

Successfully Created Spark Session


## Methods to get transactions from Etherscan.io API


### You will want some try catch logic to make sure that you are not needlessly pinging Etherscan.io

In [83]:
def read_api_key():
  api_key_json = open(r'/content/drive/MyDrive/etherScan_apiKey.json') # read in my private Etherscan.io API key from my Google Drive
  etherscan_api_key =json.load(api_key_json)['key']
  api_key_json.close()
  return etherscan_api_key

ETHERSCAN_API_KEY = read_api_key() # constant to uppercase

def query_normal_transactions(wallet_address, startblock=0, endblock=99999999) -> list: # returning a list here might be inefficient. 
  """
    Ping Etherscan.io and get the most recent 10k transactions for this wallet.
  """

  api_request_text =f'https://api.etherscan.io/api?module=account&action=txlist&address={wallet_address}&startblock={startblock}&endblock={endblock}&sort=asc&apikey={ETHERSCAN_API_KEY}'
              
  response = requests.get(api_request_text) # Need to throttle this to 5 calls a second
  api_response_text = json.loads(response.text) 
  # error handling to make sure that the call is going through
  if api_response_text['message'] != 'OK':
    print(f'you got an error at address={wallet_address} \nstartblock= {startblock}\nendblock={endblock}')
  else:
    transaction_list = api_response_text['result']     
    simplified_transactions = parse_normal_transactions(transaction_list=transaction_list)
    column_names = ['to_address',
                    'from_address',
                    'block_number',
                    'transaction_datetime',
                    'value_in_ether']
    df = pd.DataFrame(simplified_transactions,
                   columns=column_names)
    # Exclude all transactions where the pool is not the sender
    df, largest_block =clean_transaction_dataframe(df, wallet_address)
    df = spark.createDataFrame(df)
    return df, largest_block


def parse_normal_transactions(transaction_list: list):
    """
      This takes the list of 'normal' transactions and rewrites into a list of tuples that is easy to write to a file.

    :param transaction_list: A list of dictionary objects. Each dictionary is a single transaction.
    :return:
        simple_transactions: a list of tuples storing (to_address, from_address, Block number, DateTime, Value in Ether)
                                      Data types are: (String, String, int, DateTime, float)
    """
    simplified_transactions =[]

    for trans_dict in transaction_list: # trans is a dictionary object
        to_address = trans_dict['to']
        from_address = trans_dict['from']
        block_number = int(trans_dict['blockNumber'])
        # trans['value'] gets the value in wei
        value_in_wei =int(trans_dict['value'])
        value_in_ether = float(value_in_wei) / math.pow(10, 18)
        transaction_datetime = datetime.datetime.fromtimestamp(int(trans_dict['timeStamp']))
        simplified_transaction = (to_address,
                                  from_address,
                                  block_number,
                                  transaction_datetime,
                                  value_in_ether)

        simplified_transactions.append(simplified_transaction)

    return simplified_transactions

def clean_transaction_dataframe(df:pd.DataFrame, wallet_address):
  """
    Take in the transaction data from query_normal_transactions()
    Remove all records where the wallet_address is not in the from address.
      Only track where the wallet was the sender not the reciver. 
    Determine the largest block number.
    Remove all transaction records for that block. 
    Return the cleaned df and the block number of what you excluded
  """
  df = df[df['from_address'] == wallet_address]
  largest_block = df['block_number'].max()
  df = df[df['block_number'] != largest_block]
  return df, largest_block


## Basic tests on the ping method. 

In [65]:

parker_wallet = '0x76fb6d38f28c44a13380220df21363bd7af45ee1'
ethan_wallet = '0xceb4d0ca821420cf2553b9e244f6b52364613f94'
ethermine_wallet = '0xea674fdde714fd979de3edf0f56aa9716b898ec8'
df, largest_block = query_normal_transactions(wallet_address=ethermine_wallet)
print(largest_block)

df.head(10)

1271741


,to_address,from_address,block_number,transaction_datetime,value_in_ether
0,0x1f14c8687f345f99c99a5d169ffa153af7b89840,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1191228,2016-03-21 15:42:27,1.112170
1,0x1f14c8687f345f99c99a5d169ffa153af7b89840,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1192191,2016-03-21 19:34:34,1.127991
2,0xa63f4afb1812b17bc011cb516bec57e7009641a9,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1192191,2016-03-21 19:34:34,1.437429
3,0x3d1c1b034ed3495e962caadcb322e0cefb2d24cb,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1192191,2016-03-21 19:34:34,1.933763
4,0xb202d9a6314b1494ac463f152f551f7c1c577843,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1192191,2016-03-21 19:34:34,1.328998
5,0x3d31a7cdbf1b9952c1c4d567b97c54fc354e6a7b,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1194607,2016-03-22 05:12:53,1.457546
6,0xb202d9a6314b1494ac463f152f551f7c1c577843,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1194608,2016-03-22 05:13:04,1.552393
7,0x1f14c8687f345f99c99a5d169ffa153af7b89840,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1194608,2016-03-22 05:13:04,1.977245
8,0x09d8f6b6ae4c1f1c769511dcfbbae39edb9b8c2e,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1194609,2016-03-22 05:13:08,1.136996
9,0x86254a855575d0558ae8915e809c37ab8c4c6be4,0xea674fdde714fd979de3edf0f56aa9716b898ec8,1194610,2016-03-22 05:13:22,1.033290


### I think the right approach is like so


Start at start_block = 0 and end block =999999999

In [90]:
# Get all the transactions from ethermine into a pyspark
ethermine_transaction_rdd, largest_block = query_normal_transactions(wallet_address=ethermine_wallet)
# At this point you have a couple of things. 

# transaction_rdd is a cleaned rdd of the transactions from ethermine
# largest block is the next starting block.

# 11 calls takes 51 seconds when you .count each call
# the .count() is marginally slower. 3 seconds on 11 calls. 

for i in range(10):
  next_chunk_df, largest_block = query_normal_transactions(wallet_address=ethermine_wallet, startblock=largest_block)
  ethermine_transaction_rdd = ethermine_transaction_rdd.union(next_chunk_df)
  print(f'call {i}') # the .count is marginal. I don't think I need to include the 5 second throttle. because the code is already slow. 
  #print(f'number of rows in the transaction df:{ethermine_transaction_rdd.count()}') # might be a lot faster if you dont do .count() here.




1271741
call 0
call 1
call 2
call 3
call 4
call 5
call 6
call 7
call 8
call 9
1271741
call 0
call 1
call 2
call 3
call 4


KeyboardInterrupt: ignored